In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import librosa
import tensorflow as tf
import glob
import numpy as np

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [3]:
train = glob.glob('spectrogram-train/*.npy')
x = []
for fpath in train:
    fpath = train[-1].split('/')[1]
    splitted = fpath.split('-')
    if len(splitted) == 2:
        splitted[1] = splitted[1].split('.')[1]
        fpath = splitted[0] + '.' + splitted[1]
    fpath = fpath.replace('.npy','.wav')
    x.append('data/' + fpath)

In [4]:
augment = glob.glob('augment/*.wav')
x.extend(augment)

In [5]:
test_ = glob.glob('spectrogram-test/*.npy')
test = []
for t in test_:
    f = t.split('/')[1].replace('.npy', '.wav')
    test.append('data/'+f)

In [ ]:
from tqdm import tqdm

X = []
for i in tqdm(range(len(x))):
    y, sr = librosa.load(x[i], sr = 16000)
    X.append(y)

  2%|▏         | 501/29524 [00:32<30:28, 15.87it/s] 

In [ ]:
features = [(512, 10, 5), (512, 8, 4), (512, 4, 2), (512, 4, 2), (512, 4, 2), (512, 1, 1), (512, 1, 1)]
aggs = [(512, 2, 1), (512, 3, 1), (512, 4, 1), (512, 5, 1), (512, 6, 1), (512, 7, 1), (512, 8, 1), (512, 9, 1), 
 (512, 10, 1), (512, 11, 1), (512, 12, 1), (512, 13, 1)]
num_negatives = 10
prediction_steps = 12
learning_rate = 1e-5

In [ ]:
import math

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta


def cnn_block(x, hidden_dim, kernel_size, strides):
    x =  tf.layers.conv1d(inputs = x,
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          strides = strides)
    x = layer_norm(x)
    x = tf.nn.relu(x)
    return x

def cnn_aggregator(x, hidden_dim, kernel_size, strides):
    ka = kernel_size // 2
    kb = ka - 1 if kernel_size % 2 == 0 else ka
    pad = tf.zeros([tf.shape(x)[0], kb + ka, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          strides = strides)
    x = layer_norm(x)
    x = tf.nn.relu(x)
    return x

def negative_sample(y):
    bsz = tf.shape(y)[0]
    tsz = tf.shape(y)[1]
    fsz = tf.shape(y)[2]
    y = tf.transpose(y, [2, 0, 1])
    y = tf.reshape(y, (fsz, -1))
    neg_idxs = tf.random_uniform((bsz, num_negatives * tsz), minval=0, maxval=tsz, dtype=tf.int32)
    neg_idxs = tf.reshape(neg_idxs, [-1])
    negs = tf.gather(y, neg_idxs, axis=1)
    negs = tf.reshape(negs, (fsz, bsz, num_negatives, tsz))
    negs = tf.transpose(negs, [2, 1, 3, 0])
    return negs
    

class Model:
    def __init__(self):
        self.X = tf.placeholder(tf.float32, (None, None))
        feature = tf.expand_dims(self.X, axis = 2)
        
        for no, f in enumerate(features):
            size_layers = f[0]
            kernel_size = f[1]
            strides = f[2]
            with tf.variable_scope('feature_%d'%no):
                feature = cnn_block(feature, size_layers, kernel_size, strides)
        
        x = tf.identity(feature)
        for no, f in enumerate(aggs):
            size_layers = f[0]
            kernel_size = f[1]
            strides = f[2]
            with tf.variable_scope('agg_%d'%no):
                x = cnn_aggregator(x, size_layers, kernel_size, strides)
                
        jin = 0
        rin = 0
        for _, k, stride in features:
            if rin == 0:
                rin = k
            rin = rin + (k - 1) * jin
            if jin == 0:
                jin = stride
            else:
                jin *= stride
        offset = math.ceil(rin / jin)

        offset = int(offset)
        
        self.logits = x # X
        self.targets = feature # Y
        self.negatives = negative_sample(self.targets)
        
        y = tf.expand_dims(self.targets, axis = 0)
        targets = tf.concat([y, self.negatives], axis = 0)
        b = tf.shape(targets)[0]
        x = tf.expand_dims(self.logits, axis = -1)
        
        x = tf.layers.conv2d_transpose(x, prediction_steps, (1, 1))
        x = tf.expand_dims(x, axis = 0) 
        x = tf.tile(x, [b, 1, 1, 1, 1])
        
        copies = tf.shape(x)[0]
        bsz = tf.shape(x)[1]
        tsz = tf.shape(x)[2]
        dim = tf.shape(x)[3]
        steps = tf.shape(x)[4]
        self.o = x
        self.p = targets
        
        steps = tf.math.minimum(steps, tsz - offset)
        predictions = tf.zeros(bsz * copies * (tsz - offset + 1) * \
                               steps - ((steps + 1) * steps // 2) * copies * bsz)
        labels = tf.zeros_like(predictions)
        
        def body(i, start, end, predictions, labels):
            offset_ = i + offset
            end = start + (tsz - offset_) * bsz * copies
            pos_num = (end - start) // copies
            s = tf.reduce_sum((x[:, :, :-offset_, :, i] * targets[:, :, offset_:, :]), axis = 3)
            s = tf.reshape(s, [-1])
            s = tf.pad(s, [[start, tf.shape(predictions)[0] - (start + tf.shape(s)[0])]])
            predictions = tf.add(predictions, s)
            l = tf.ones((pos_num))
            l = tf.pad(l, [[start, tf.shape(labels)[0] - (start + pos_num)]])
            labels = tf.add(labels, l)
            return i + 1, end, end, predictions, labels

        def condition(i, start, end, predictions, labels):
            return i < steps

        ranged = tf.Variable(tf.constant(0))
        _, _, _, predictions, labels = tf.while_loop(condition, body, [0, 0, 0, predictions, labels])
        self.predictions = predictions
        self.labels = labels
        
        self.cost = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=self.labels,
            logits=self.predictions,
        )
        self.cost = tf.reduce_mean(self.cost)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
sess.run(tf.global_variables_initializer())

In [ ]:
l, t, o, p, q = sess.run([model.logits, model.targets, model.o, model.predictions, model.labels], 
                         feed_dict = {model.X: resampled})
l.shape, t.shape, o.shape, p.shape

In [ ]:
sess.run(model.cost, feed_dict = {model.X: resampled})